<a href="https://colab.research.google.com/github/brycegriffin1977-bot/AI-Gourmet/blob/main/Stock_Market_Momentum_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

# --- Configuration ---
# NOTE: Replace this list with the tickers you want to analyze.
# A small list is used for a quick demonstration. Analyzing "all stocks"
# requires access to professional data feeds and significant computation.
TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'JPM', 'NVDA']
START_DATE = '2023-01-01'
END_DATE = pd.to_datetime('today').strftime('%Y-%m-%d')

# Moving Average periods for the Golden Cross/Death Cross strategy
SMA_SHORT = 50
SMA_LONG = 200
RECENT_RETURN_DAYS = 5 # Used to rank the top predictions (short-term momentum)

def analyze_and_backtest(ticker):
    """
    Downloads historical data, applies a moving average strategy, and
    calculates backtested performance and today's signal.
    """
    try:
        # 1. Data Download
        data = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False)
        if data.empty:
            print(f"--- ERROR: Could not retrieve data for {ticker}. Skipping. ---")
            return None, None

        # 2. Calculate Simple Moving Averages (SMAs)
        data[f'SMA_{SMA_SHORT}'] = data['Close'].rolling(window=SMA_SHORT).mean()
        data[f'SMA_{SMA_LONG}'] = data['Close'].rolling(window=SMA_LONG).mean()

        # 3. Strategy Logic (Golden Cross Momentum)
        # Signal 1: Short SMA crosses above Long SMA (Long-term trend confirmation)
        data['Signal_Cross'] = np.where(data[f'SMA_{SMA_SHORT}'] > data[f'SMA_{SMA_LONG}'], 1, 0)

        # Signal 2: Current price is above short-term SMA (Short-term strength)
        data['Signal_Price'] = np.where(data['Close'] > data[f'SMA_{SMA_SHORT}'], 1, 0)

        # Final Buy Signal: Both signals must be active
        data['Buy_Signal'] = np.where((data['Signal_Cross'] == 1) & (data['Signal_Price'] == 1), 1, 0)

        # Calculate daily returns
        data['Daily_Return'] = data['Close'].pct_change()

        # 4. Backtest Performance (Simple Buy and Hold vs. Strategy)

        # Calculate strategy returns:
        # When Buy_Signal is 1, return is Daily_Return, otherwise 0 (out of the market)
        data['Strategy_Return'] = data['Daily_Return'] * data['Buy_Signal'].shift(1)

        # Cumulative Returns
        data['Cumulative_Buy_Hold'] = (1 + data['Daily_Return']).cumprod()
        data['Cumulative_Strategy'] = (1 + data['Strategy_Return']).cumprod()

        # Extract total returns
        if data['Cumulative_Buy_Hold'].iloc[-1] is not np.nan:
            buy_hold_return = (data['Cumulative_Buy_Hold'].iloc[-1] - 1) * 100
        else:
            buy_hold_return = 0.0

        if data['Cumulative_Strategy'].iloc[-1] is not np.nan:
            strategy_return = (data['Cumulative_Strategy'].iloc[-1] - 1) * 100
        else:
            strategy_return = 0.0

        # 5. Today's Prediction Signal
        # A stock is favorable if the buy signal is active on the last available day
        today_signal = data['Buy_Signal'].iloc[-1]

        # Calculate recent momentum for ranking
        recent_momentum = data['Daily_Return'].tail(RECENT_RETURN_DAYS).sum() * 100

        results = {
            'Ticker': ticker,
            'Buy_Hold_Return': buy_hold_return,
            'Strategy_Return': strategy_return,
            'Is_Favorable': bool(today_signal),
            'Recent_Momentum_5D': recent_momentum
        }

        return results, data.tail(1)

    except Exception as e:
        print(f"--- ERROR processing {ticker}: {e} ---")
        return None, None

def main():
    print(f"--- Stock Market Momentum Analyzer: {END_DATE} ---")
    print(f"Analyzing {len(TICKERS)} tickers using data from {START_DATE} to today.")
    print(f"Strategy: Golden Cross Momentum (50-day SMA > 200-day SMA AND Close > 50-day SMA)\n")

    all_results = []

    # Process all tickers
    for ticker in TICKERS:
        results, _ = analyze_and_backtest(ticker)
        if results:
            all_results.append(results)

    if not all_results:
        print("No successful results to process.")
        return

    df_results = pd.DataFrame(all_results)

    # 1. Backtesting Summary
    print("=========================================")
    print("        BACKTEST PERFORMANCE SUMMARY")
    print("=========================================")

    # Format and print the results table
    df_summary = df_results[['Ticker', 'Buy_Hold_Return', 'Strategy_Return']]
    df_summary['Buy_Hold_Return'] = df_summary['Buy_Hold_Return'].map('{:,.2f}%'.format)
    df_summary['Strategy_Return'] = df_summary['Strategy_Return'].map('{:,.2f}%'.format)
    print(df_summary.to_string(index=False))

    print("\n*Note: This simulation assumes you only enter the market when the Buy Signal is active.*")

    # 2. Next-Day Prediction/Recommendation
    print("\n=========================================")
    print("  NEXT-DAY JUMP/ADVANTAGE PREDICTION")
    print("=========================================")

    # Filter for stocks with an active 'Is_Favorable' signal
    df_predictions = df_results[df_results['Is_Favorable'] == True].sort_values(
        by='Recent_Momentum_5D', ascending=False
    )

    if df_predictions.empty:
        print("\nNo stocks currently meet the momentum criteria for a jump/advantage.")
    else:
        print(f"Found {len(df_predictions)} favorable stocks based on strategy rules:")

        df_predictions['Recent_Momentum_5D'] = df_predictions['Recent_Momentum_5D'].map('{:,.2f}%'.format)

        print(df_predictions[['Ticker', 'Recent_Momentum_5D']].to_string(index=False))

        top_pick = df_predictions.iloc[0]['Ticker']
        print(f"\n🥇 TOP PICK FOR NEXT DAY ADVANTAGE (Highest Recent Momentum): {top_pick}")
        print("\n*Prediction is based on current long-term trend and short-term strength.*")

if __name__ == "__main__":
    main()

--- Stock Market Momentum Analyzer: 2025-10-28 ---
Analyzing 7 tickers using data from 2023-01-01 to today.
Strategy: Golden Cross Momentum (50-day SMA > 200-day SMA AND Close > 50-day SMA)

--- ERROR processing AAPL: Operands are not aligned. Do `left, right = left.align(right, axis=1, copy=False)` before operating. ---
--- ERROR processing MSFT: Operands are not aligned. Do `left, right = left.align(right, axis=1, copy=False)` before operating. ---
--- ERROR processing GOOGL: Operands are not aligned. Do `left, right = left.align(right, axis=1, copy=False)` before operating. ---
--- ERROR processing AMZN: Operands are not aligned. Do `left, right = left.align(right, axis=1, copy=False)` before operating. ---
--- ERROR processing TSLA: Operands are not aligned. Do `left, right = left.align(right, axis=1, copy=False)` before operating. ---


/tmp/ipython-input-2604200577.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False)
/tmp/ipython-input-2604200577.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False)
/tmp/ipython-input-2604200577.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False)
/tmp/ipython-input-2604200577.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False)
/tmp/ipython-input-2604200577.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False)
/tmp/ipython-input-2604200577.py:25: Fut

--- ERROR processing JPM: Operands are not aligned. Do `left, right = left.align(right, axis=1, copy=False)` before operating. ---
--- ERROR processing NVDA: Operands are not aligned. Do `left, right = left.align(right, axis=1, copy=False)` before operating. ---
No successful results to process.


/tmp/ipython-input-2604200577.py:25: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False)
